In [41]:
import pandas as pd
import numpy as np
import scipy.stats

In [42]:
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    print(f'AUC model 1: {aucs[0]}, AUC model 2: {aucs[-1]}')
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

In [ ]:
#all_preds = pd.read_csv("/fast/grantn2/BBBPermeability/analysis_outputs/all_preds_long_format.csv")
all_preds = pd.read_csv("Outputs_Sept2025/all_preds_long_format.csv")
    # col1 "trained_on" --> [B3DB, NeuTox]
    # col2 "test_set" --> [B3DB, NeuTox, BBBP]
    # col3 "feature_type" --> [Descriptors, Embeddings, Combined]
    # col4 "fold" --> [0, 1, 2, 3, 4]
    # col5 "classifier" --> [RF, SVM, LR, MLP]
    # col6 "label"
    # col7 "prob"
    # col8 "pred"
ensemble_preds = pd.read_csv("Outputs_Sept2025/all_preds_long_format.csv")
# Comparisons I want to do: keep trained_on and test_set the same across all comparisons
# Our model ---> Classifer == MLP & feature_type == Combined

# 1. Combined features vs Descriptors

In [44]:
training_set = "B3DB"
test_set = "B3DB" 


# Base Model: MLP + Combined features
base_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Combined") & (all_preds["classifier"] == "MLP")]
base_model_labels = base_model["label"].values
base_model_probs = base_model["prob"].values

# Descriptors Model: MLP + Descriptors
desc_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Descriptors") & (all_preds["classifier"] == "MLP")]
desc_model_probs = desc_model["prob"].values
desc_model_labels = desc_model["label"].values

# Encoded Feat Model: MLP + Encoded Features
encoded_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Embeddings") & (all_preds["classifier"] == "MLP")]
encoded_model_probs = encoded_model["prob"].values
encoded_model_labels = encoded_model["label"].values

# SVM Model: SVM + Combined features
svm_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Combined") & (all_preds["classifier"] == "SVM")]
svm_model_probs = svm_model["prob"].values
svm_model_labels = svm_model["label"].values

# Logistic Regression Model: LR + Combined features
lr_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Combined") & (all_preds["classifier"] == "LR")]
lr_model_probs = lr_model["prob"].values
lr_model_labels = lr_model["label"].values

# RF Model: RF + Combined features
rf_model = all_preds[(all_preds["trained_on"] == training_set) & (all_preds["test_set"] == test_set) & (all_preds["feature_type"] == "Combined") & (all_preds["classifier"] == "RF")]
rf_model_probs = rf_model["prob"].values
rf_model_labels = rf_model["label"].values

# Multi-Head Model: Desc head, encoder head, combined head
mh_model = ensemble_preds[(ensemble_preds["trained_on"] == training_set) & (ensemble_preds["test_set"] == test_set) & (ensemble_preds["feature_type"] == "All") & (ensemble_preds["classifier"] == "Ensemble")]
mh_model_probs = mh_model["prob"].values
mh_model_labels = mh_model["label"].values


In [45]:
# Assert that labels are the same across all models
assert np.array_equal(base_model_labels, desc_model_labels)
assert np.array_equal(base_model_labels, encoded_model_labels)
assert np.array_equal(base_model_labels, svm_model_labels)
assert np.array_equal(base_model_labels, lr_model_labels)
assert np.array_equal(base_model_labels, rf_model_labels)
assert np.array_equal(base_model_labels, mh_model_labels)


In [46]:
# Assert that labels are the same across all models


# Compare everything to the mH_model

assert np.array_equal(mh_model_labels, desc_model_labels)
assert np.array_equal(mh_model_labels, encoded_model_labels)
assert np.array_equal(mh_model_labels, svm_model_labels)
assert np.array_equal(mh_model_labels, lr_model_labels)
assert np.array_equal(mh_model_labels, rf_model_labels)
assert np.array_equal(mh_model_labels, base_model_labels)


In [47]:
Models = {
    "Descriptors Model": desc_model_probs,
    "Encoded Features Model": encoded_model_probs,
    "SVM Model": svm_model_probs,
    "Logistic Regression Model": lr_model_probs,
    "RF Model": rf_model_probs,
    "Combined Model": base_model_probs
}

import os
import csv

output_path = "/fast/grantn2/BBBPermeability/analysis_outputs/DeLong_test_results/Oct2025"
os.makedirs(output_path, exist_ok=True)
output_file = os.path.join(output_path, f"DeLong_train_{training_set}_test_{test_set}.csv")


for model_name, model_predictions in Models.items():
    # Perform DeLong test
    log_p_value = delong_roc_test(mh_model_labels, mh_model_probs, model_predictions)

    # Convert log p-value to standard p-value
    log_p_value = np.exp(np.log(10)*log_p_value)

    # Print the results
    print(f"Comparison: MLP Ensemble (Base Model) vs {model_name}")
    print(f"Log p-value: {log_p_value.item():.3f}")
    # print(f"p-value: {p_value.item():.3e}")
    print("-")


with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Comparison", "Log p-value"])
    for model_name, model_predictions in Models.items():
        log_p_value = delong_roc_test(mh_model_labels, mh_model_probs, model_predictions)
        log_p_value = np.exp(np.log(10)*log_p_value)
        writer.writerow([f"MLP Ensemble (Base Model) vs {model_name}", log_p_value.item()])



AUC model 1: 0.9868697799183075, AUC model 2: 0.9602921340608424
Comparison: MLP Ensemble (Base Model) vs Descriptors Model
Log p-value: 0.000
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9856743050051819
Comparison: MLP Ensemble (Base Model) vs Encoded Features Model
Log p-value: 0.520
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9396394714381515
Comparison: MLP Ensemble (Base Model) vs SVM Model
Log p-value: 0.000
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9479877918673414
Comparison: MLP Ensemble (Base Model) vs Logistic Regression Model
Log p-value: 0.000
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9919193516429918
Comparison: MLP Ensemble (Base Model) vs RF Model
Log p-value: 0.019
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9850894272389199
Comparison: MLP Ensemble (Base Model) vs Combined Model
Log p-value: 0.204
-
AUC model 1: 0.9868697799183075, AUC model 2: 0.9602921340608424
AUC model 1: 0.9868697799183075, AUC model 2: 0.9856743050051819
AUC mod

In [48]:
# Making ROC curves for each
#for model_name, model_probs in Models.items():
#    from sklearn.metrics import roc_curve, auc
##    import matplotlib.pyplot as plt

 #   fpr, tpr, _ = roc_curve(base_model_labels, model_probs)
  ##
   ## plt.figure()
    #plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
   # plt.xlabel('False Positive Rate')
   # plt.ylabel('True Positive Rate')
   # plt.title(f'Receiver Operating Characteristic - {model_name}')
   # plt.legend(loc="lower right")
   # plt.savefig(f"/fast/grantn2/BBBPermeability/Outputs_Sept2025/ROC_Curve/{training_set}_train_{test_set}_{model_name.replace(' ', '_')}.png")
   # plt.show()
   #plt.close()